# Azureの補完の例
この例では、Azureエンドポイントを使用して補完を実行するために必要なすべての操作について説明します。 \
ただし、この例は補完に焦点を当てており、APIを使用した他の操作にも触れています。この例は、単純な操作を示すためのクイックな方法として提供されており、チュートリアルではありません。

In [ ]:
import openai

## セットアップ
以下のセクションが正しく機能するようにするために、いくつかの設定を行う必要があります。まず、api_baseとapi_versionを設定します。api_baseは、https://portal.azure.com にアクセスし、リソースを見つけて、「リソース管理」->「キーとエンドポイント」の下で「エンドポイント」の値を確認することで見つけることができます。

In [ ]:
openai.api_version = '2023-05-15'
openai.api_base = '' # Please add your endpoint here

次に、api_typeとapi_keyを設定する必要があります。キーをポータルから取得するか、Microsoft Active Directory認証を介して取得することができます。これに応じて、api_typeは「azure」または「azure_ad」のいずれかになります。

### セットアップ: ポータル
まず、ポータルからキーを取得する方法を見てみましょう。https://portal.azure.com にアクセスし、リソースを見つけて、「リソース管理」->「キーとエンドポイント」の下で「キー」の値を確認します。

In [ ]:
openai.api_type = 'azure'
openai.api_key = ''  # Please add your api key here

### (オプション) セットアップ: Microsoft Active Directory認証
次に、Microsoft Active Directory認証を使用してキーを取得する方法を見てみましょう。ポータルからのキーの代わりにActive Directory認証を使用したい場合は、次のコードのコメントを外してください。

In [ ]:
# from azure.identity import DefaultAzureCredential

# default_credential = DefaultAzureCredential()
# token = default_credential.get_token("https://cognitiveservices.azure.com/.default")

# openai.api_type = 'azure_ad'
# openai.api_key = token.token

トークンは一定期間有効であり、その後期限切れになります。有効なトークンがすべてのリクエストと共に送信されるようにするために、期限切れのトークンを更新するには、requests.authにフックすることができます。

In [ ]:
import typing
import time
import requests
if typing.TYPE_CHECKING:
    from azure.core.credentials import TokenCredential

class TokenRefresh(requests.auth.AuthBase):

    def __init__(self, credential: "TokenCredential", scopes: typing.List[str]) -> None:
        self.credential = credential
        self.scopes = scopes
        self.cached_token: typing.Optional[str] = None

    def __call__(self, req):
        if not self.cached_token or self.cached_token.expires_on - time.time() < 300:
            self.cached_token = self.credential.get_token(*self.scopes)
        req.headers["Authorization"] = f"Bearer {self.cached_token.token}"
        return req

session = requests.Session()
session.auth = TokenRefresh(default_credential, ["https://cognitiveservices.azure.com/.default"])

openai.requestssession = session

## デプロイメント
このセクションでは、text-davinci-002モデルを使用してデプロイメントを作成し、そのデプロイメントを使用して補完を作成します。

### デプロイメント: 手動で作成
ポータルのリソース内で、「リソース管理」->「モデルのデプロイ」の下にある「新しいデプロイメント」を選択し、text-davinci-002をモデルとして選択して新しいデプロイメントを作成します。

In [ ]:
deployment_id = '' # Fill in the deployment id from the portal here

### 補完
さて、サンプルの補完をデプロイメントに送信しましょう。

In [ ]:
prompt = "The food was delicious and the waiter"
completion = openai.Completion.create(deployment_id=deployment_id,
                                     prompt=prompt, stop=".", temperature=0)
                                
print(f"{prompt}{completion['choices'][0]['text']}.")